#Title

##Muslim moving to Japan. 
### Problem Statement 
* to seek area with the easiest Mosque Access
* to seek area where it is easy to cater to special muslim food dietary 
* to find an area with an easiest access to trains and transportation

### Method
* using set of list and location manually
* get geopy location of those location
* get foresquare Api of  Mosque, Train station and Halal food 



## Install Dependency

In [1]:
!pip install geopy
!pip install folium==0.5.0

## Set Imports

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
#from pandas.io.json import json_normalize
from pandas import json_normalize
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


## Create dataframe of neighbourhood, longitude and latitude



In [3]:
#url="https://raw.githubusercontent.com/netnavi20x5/Coursera_Capstone/master/data/japan_list_neighbour.txt"
url="data/Tokyo_ward.txt"
neighbour_list=pd.read_csv(url,header = None,sep='|')
neighbour_list=neighbour_list.rename(columns = {0: 'neighbour'}, inplace = False)
neighbour_list

,neighbour
0,"Adachi, Tokyo"
1,"Arakawa, Tokyo"
2,"Bunkyo, Tokyo"
3,"Chiyoda, Tokyo"
4,"Chuo, Tokyo"
5,"Edogawa, Tokyo"
6,"Itabashi, Tokyo"
7,Katsushika
8,"Kita, tokyo"
9,"Koto , Tokyo"


In [4]:
def geolocation_array(address):
  address = address
  geolocator = Nominatim(user_agent="foursquare_agent")
  location = geolocator.geocode(address)
  if location is not None:
    latitude = location.latitude
    longitude = location.longitude
    #print("output :",address," Geocode ",latitude, longitude)
    return [address,latitude,longitude]


In [5]:
df_neigbbourhood = pd.DataFrame(columns=['neighbourhood','longitude','latitude'])
count=0
for i in neighbour_list['neighbour']:

  if geolocation_array(i) is not None:
    df_neigbbourhood=df_neigbbourhood.append(pd.DataFrame([geolocation_array(i)],columns=['neighbourhood','longitude','latitude']), ignore_index=True)
  else:
    print("No location found for ",i," ingnoring location ")
  if count%5==0:
    print("processing : ",round(count/neighbour_list['neighbour'].size*100,2),'%')
  count=count+1
df_neigbbourhood

processing :  0.0 %
processing :  21.74 %
processing :  43.48 %
processing :  65.22 %
processing :  86.96 %


,neighbourhood,longitude,latitude
0,"Adachi, Tokyo",35.783703,139.795319
1,"Arakawa, Tokyo",35.737529,139.781310
2,"Bunkyo, Tokyo",35.718810,139.744732
3,"Chiyoda, Tokyo",35.682112,139.765674
4,"Chuo, Tokyo",35.666255,139.775565
5,"Edogawa, Tokyo",35.678278,139.871091
6,"Itabashi, Tokyo",35.774143,139.681209
7,Katsushika,35.751733,139.863816
8,"Kita, tokyo",35.755838,139.736687
9,"Koto , Tokyo",35.649154,139.812790


### Define Foursquare Credentials and Version

In [6]:
CLIENT_ID = 'KRX03NAYCVNWTSOWCDW3C5RJW0GQIQXPCLG1FZ4SG4Y4CV52' # your Foursquare ID
CLIENT_SECRET = '2Y0LD2DJIKTA3MULS3WK41AD1VCPNZYRHHVPS0NXKGEMK5XW' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KRX03NAYCVNWTSOWCDW3C5RJW0GQIQXPCLG1FZ4SG4Y4CV52
CLIENT_SECRET:2Y0LD2DJIKTA3MULS3WK41AD1VCPNZYRHHVPS0NXKGEMK5XW


### Get data from Foresquare API
Mosque Radius = 5 KM
Halal Food Radius = 5 KM
Train station = 1KM

Reasonsing 
Mosque and Halal food can use trains and busses

In [7]:
#forsquare URL function Generator
def forsquare_query(latitude, longitude, search_query, radius, LIMIT):
    api_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, longitude,latitude,  VERSION, search_query, radius, LIMIT)
    return api_url
def forsquare_query_json(url):
    results = requests.get(url).json()
    return results

In [8]:
LIMIT = 100
#custom list where mosque is 5km halal Shops is 5 km and Train station within 1km Radius
prefered_search_query=[["Mosque",5000],["Halal",5000],["Train_Station",1000]]


In [15]:
df_neighbour_data=pd.DataFrame()
for i in range(len(df_neigbbourhood.index)):
    df_neigbbourhood.iloc[i]
    latitude=df_neigbbourhood.iloc[i]['latitude']
    longitude=df_neigbbourhood.iloc[i]['longitude']
    for items_no in range (len(prefered_search_query)):
        #print (prefered_search_query[lines])
        search_query=prefered_search_query[items_no][0]
        radius=prefered_search_query[items_no][1]
        url=forsquare_query(latitude, longitude, search_query, radius, LIMIT)
        #print(url)
        results = requests.get(url).json()
        venues=results['response']['venues']
        #print (venues)
        df_data=json_normalize(venues)
        df_data['categories_2']=search_query
        df_data['neighbourhood']=df_neigbbourhood.iloc[i]['neighbourhood']
        df_data['latitude']=df_neigbbourhood.iloc[i]['latitude']
        df_data['longitude']=df_neigbbourhood.iloc[i]['longitude']
        df_neighbour_data=df_neighbour_data.append(df_data,ignore_index=True)
        

In [41]:
df_neighbour_data

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,...,location.state,location.country,location.formattedAddress,categories_2,neighbourhood,latitude,longitude,location.crossStreet,location.neighborhood,venuePage.id
0,4dbe7507ec8eb49ed33434c4,Jamea Mosque Yashio,"[{'id': '4bf58dd8d48988d138941735', 'name': 'M...",v-1612081983,False,Ukizuka 649-1,35.797812,139.828354,"[{'label': 'display', 'lat': 35.79781193073883...",3371.0,...,埼玉県,日本,"[Ukizuka 649-1, 八潮市, 埼玉県, 340-0835, 日本]",Mosque,"Adachi, Tokyo",139.795319,35.783703,NaN,NaN,NaN
1,5feb0117116a6a1b384edc3d,Ds Halal Food & Spice Shop,"[{'id': '56aa371be4b08b9a8d573550', 'name': 'F...",v-1612081983,False,NaN,35.815323,139.802120,"[{'label': 'display', 'lat': 35.815323, 'lng':...",3573.0,...,埼玉県,日本,"[Sōka, 埼玉県, 340-0022, 日本]",Halal,"Adachi, Tokyo",139.795319,35.783703,NaN,NaN,NaN
2,4ee730de29c220f9743d1e37,アルカラム,"[{'id': '52e81612bcbc57f1066b79f8', 'name': 'P...",v-1612081983,False,中央1-8-10,35.821071,139.838484,"[{'label': 'display', 'lat': 35.82107147932116...",5700.0,...,埼玉県,日本,"[中央1-8-10, 八潮市, 埼玉県, 340-0816, 日本]",Halal,"Adachi, Tokyo",139.795319,35.783703,NaN,NaN,NaN
3,4b7a3e24f964a5205c272fe3,Nishiarai Station (TS13) (西新井駅),"[{'id': '4bf58dd8d48988d129951735', 'name': 'T...",v-1612081984,False,西新井栄町2-1-1,35.777208,139.790458,"[{'label': 'display', 'lat': 35.777208, 'lng':...",845.0,...,東京都,日本,"[西新井栄町2-1-1, 足立区, 東京都, 123-0843, 日本]",Train_Station,"Adachi, Tokyo",139.795319,35.783703,NaN,NaN,NaN
4,4fae17f0e4b0664ab77690a3,Asakusa Mosque (浅草マスジド),"[{'id': '4bf58dd8d48988d138941735', 'name': 'M...",v-1612081984,False,東浅草1-9-12,35.721756,139.801573,"[{'label': 'display', 'lat': 35.7217559711829,...",2536.0,...,東京都,日本,"[東浅草1-9-12, 台東区, 東京都, 111-0025, 日本]",Mosque,"Arakawa, Tokyo",139.781310,35.737529,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648,5dfb5b5d2fc4210008f40fd3,Pokémon Card Station (ポケモンカードステーション),"[{'id': '4bf58dd8d48988d1fb941735', 'name': 'H...",v-1612082025,False,東池袋3-1-2,35.729019,139.719240,"[{'label': 'display', 'lat': 35.729019, 'lng':...",914.0,...,東京都,日本,"[東池袋3-1-2 (サンシャインシティアルパ 2F), 豊島区, 東京都, 日本]",Train_Station,"Toshima, Tokyo",139.714222,35.736156,サンシャインシティアルパ 2F,NaN,NaN
649,52a96e0611d2ca203d1b02af,MEGMILK STATION,"[{'id': '4bf58dd8d48988d112941735', 'name': 'J...",v-1612082025,False,NaN,35.729242,139.710674,"[{'label': 'display', 'lat': 35.729242, 'lng':...",833.0,...,NaN,日本,[日本],Train_Station,"Toshima, Tokyo",139.714222,35.736156,NaN,NaN,NaN
650,4b88c2acf964a520450c32e3,Ikebukuro Police Station (池袋警察署),"[{'id': '4bf58dd8d48988d12e941735', 'name': 'P...",v-1612082025,False,西池袋1-7-5,35.728573,139.706894,"[{'label': 'display', 'lat': 35.7285729458165,...",1072.0,...,東京都,日本,"[西池袋1-7-5, 豊島区, 東京都, 171-0021, 日本]",Train_Station,"Toshima, Tokyo",139.714222,35.736156,NaN,NaN,NaN
651,4e97c479f5b9dd852f974208,Station Conference Ikebukuro (ステーションコンファレンス池袋),"[{'id': '4bf58dd8d48988d127941735', 'name': 'C...",v-1612082025,False,西池袋1-11-1,35.728958,139.709623,"[{'label': 'display', 'lat': 35.728958, 'lng':...",902.0,...,東京都,日本,"[西池袋1-11-1 (メトロポリタンプラザビル 2F/12F), 豊島区, 東京都, 17...",Train_Station,"Toshima, Tokyo",139.714222,35.736156,メトロポリタンプラザビル 2F/12F,NaN,NaN


In [44]:
#cleaning category Field
df_neighbour_data[["id","name","categories","location.lat","location.lng","location.distance"]]
df_neighbour_category=df_neighbour_data['categories']

#create empty dataframe for the categories
df_temp_cat = pd.DataFrame(columns = ['categories.name']) 

for lvl1 in range (len(df_neighbour_data.index)):
  try:
    df_temp_cat = df_temp_cat.append({'categories.name' : df_neighbour_category[lvl1][0]['name'] },ignore_index = True) 

  except IndexError:
    df_temp_cat = df_temp_cat.append({'categories.name' : 'none' },ignore_index = True) 

df_neighbour_data_all=pd.merge(df_neighbour_data,df_temp_cat,left_index=True,right_index=True)



In [51]:
df_neighbour_data_all[['neighbourhood','categories.name','categories_2','location.distance']]

,neighbourhood,categories.name,categories_2,location.distance
0,"Adachi, Tokyo",Mosque,Mosque,3371.0
1,"Adachi, Tokyo",Food Service,Halal,3573.0
2,"Adachi, Tokyo",Pakistani Restaurant,Halal,5700.0
3,"Adachi, Tokyo",Train Station,Train_Station,845.0
4,"Arakawa, Tokyo",Mosque,Mosque,2536.0
...,...,...,...,...
648,"Toshima, Tokyo",Hobby Shop,Train_Station,914.0
649,"Toshima, Tokyo",Juice Bar,Train_Station,833.0
650,"Toshima, Tokyo",Police Station,Train_Station,1072.0
651,"Toshima, Tokyo",Conference Room,Train_Station,902.0
